# Clase 13: taller final
### Herramientas de programación para el análisis de datos | Universidad de los Andes

## 0. Program set up

Importando las librerías necesarias, si no están instaladas podemos usar<br>
**pip install matplotlib**

In [ ]:
# Pandas para manipular los DataFrames
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## 1. Consolidar datos

**Noten:** ¡Pandas nos permite cargar archivos en formato nativo de STATA (.dta)!

In [ ]:
# Cargamos los datos urbanos a DataFrames de Pandas
Upersonas = pd.read_stata('Upersonas.dta')
Uhogar = pd.read_stata('Uhogar.dta')

In [ ]:
# Cargamos los rurales a DataFrames de Pandas
# TODO importar Rpersonas y Rhogar


In [ ]:
Upersonas.head(6)

In [ ]:
Uhogar.head(6)

### Uniones horizontales internas (inner joins) de las bases Rural y Urbana
Inner porque nos interesan las observaciones que se encuentran en ambas bases, tanto personas como hogares.

llave_n16 es sólo la llave que une personas con hogares para 2016. Eso indica el diccionario de la base.

In [ ]:
Urbano = Upersonas.merge(Uhogar, on='llave_n16')
Rural = Rpersonas.merge(Rhogar, on='llave_n16')

### Base consolidada agregando verticalmente Rural y Urbano

In [ ]:
Datos = Rural.append(Urbano)

In [ ]:
# Observamos la cantidad de filas y columnas
Datos.shape

### Visualizamos

In [ ]:
Datos.head(10)

### Dejar las variables importantes

In [ ]:
Datos = Datos.rename(columns={'sexo': 'mujer'})

In [ ]:
columnas = ['hoy_fuma', 'come_paquete', 'come_fritos', 'hospital_veces', 'ehace_deporte', 'zona', 'edad', 'mujer', 'hijos_vivos', 'sp_estrato', 't_personas', 'tercil2016', 'ha_fumado', 'dias_noasistio', 'afiliacion']
Datos = Datos[columnas]

### Revisamos finalmente la base consolidada

In [ ]:
Datos.head()

In [ ]:
Datos.shape

In [ ]:
Datos.dtypes

## 2. Editar variables

### Codificar valores

Primero construimos la columna completamente numérica

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: zona es missing value
seleccionDeFilas = Datos['zona'].isnull()
# Indicamos como Rural las filas donde se cumple la selección
Datos.loc[seleccionDeFilas, 'zona'] = 0

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: zona es missing value
seleccionDeFilas = (Datos['zona']=='Urbano')
# Indicamos como Rural las filas donde se cumple la selección
Datos.loc[seleccionDeFilas, 'zona'] = 1

**Tenemos:** los siguientes valores

In [ ]:
Datos['zona'].value_counts()

*********
**Note:** En Pandas sólo es necesario asignar el tipo categórico a la columna y así la base tendrá menor tamaño y las operaciones ocurrirán con mayor velocidad. Ahora mismo, zona no es categórica.
*********

In [ ]:
Datos.dtypes

**Así que:** la asignamos categórica y generamos un diccionario

In [ ]:
Datos["zona"] = Datos["zona"].astype('category')
label_zona = dict(enumerate(Datos['zona'].cat.categories))
print(label_zona)

**Luego:** podemos editar ese diccionario como nos parezca mejor

In [ ]:
label_zona = {0: 'Rural', 1: 'Urbano'}
label_zona

### Así, para ir de números a etiquetas:

In [ ]:
Datos['zona_cat'] = Datos['zona'].map(label_zona)
Datos["zona_cat"] = Datos["zona_cat"].astype('category')

In [ ]:
Datos[['zona', 'zona_cat']]

**Note que las dos son categóricas**

In [ ]:
Datos[['zona', 'zona_cat']].dtypes

### Y, para ir de etiquetas a números:

In [ ]:
Datos['zona_num'] = Datos['zona_cat'].astype('category').cat.codes
Datos['zona_num']= Datos['zona_num'].astype('category')

In [ ]:
Datos[['zona', 'zona_cat', 'zona_num']]

In [ ]:
Datos[['zona', 'zona_cat', 'zona_num']].dtypes

### Respecto a categóricas siguentes

In [ ]:
# TODO Defina una lista llamada "columnas" que incluya las columnas 
# 'ha_fumado', 'come_paquete', 'come_fritos', 'hijos_vivos'


In [ ]:
for col in columnas:
    print(Datos[col].value_counts())
    print('missing', Datos[col].isna().sum())
    print()

### Recodificamos Sí a 1, No a 0

In [ ]:
for col in columnas:
    label_temp = dict(enumerate(Datos[col].cat.categories))
    print(label_temp)

### Las convertimos en números

In [ ]:
for col in columnas:
    Datos[col] = Datos[col].astype('category').cat.codes

In [ ]:
for col in columnas:
    print(Datos[col].value_counts())
    print('missing', Datos[col].isna().sum())
    print()

**Convirtió:** No en 1, Sí en 0, y NaN en -1

In [ ]:
for col in columnas:
    sel = (Datos[col]==1)
    Datos.loc[sel, col] = 2
    
    sel = (Datos[col]==0)
    Datos.loc[sel, col] = 1
    
    # TODO a cada columna que tenga el valor 2, de vuelta a 0 (igual que aquí arriba)
    
    
    sel = (Datos[col]==-1)
    Datos.loc[sel, col] = np.NaN

In [ ]:
for col in columnas:
    print(Datos[col].value_counts())
    print('missing', Datos[col].isna().sum())
    print()

### Ya solo sería mapearlo según una etiqueta para 0 y 1 como está en las siguientes dos celdas

### PERO: no vamos a hacerlo porque vamos a usar los valores numéricos después

In [ ]:
# No ejecutar
label_booleano = {0: 'No', 1: 'Sí'}
for col in columnas:
    Datos[col] = Datos[col].map(label_booleano)
    Datos[col] = Datos[col].astype('category')

In [ ]:
# No ejecutar
Datos[columnas]

### Y para mujer:

In [ ]:
Datos['mujer'].value_counts()

In [ ]:
Datos['mujer'].isna().sum()

### Renombramos mujer

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: es igual a 'Mujer'
seleccionDeFilas = (Datos['mujer'] == 'Mujer')
Datos.loc[seleccionDeFilas, 'mujer'] = 'Sí'

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: es igual a 'MUJER'
seleccionDeFilas = (Datos['mujer'] == 'MUJER')
Datos.loc[seleccionDeFilas, 'mujer'] = 'Sí'

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: es igual a 'Hombre'
seleccionDeFilas = (Datos['mujer'] == 'Hombre')
Datos.loc[seleccionDeFilas, 'mujer'] = 'No'

In [ ]:
# Una columna de verdaderos y falsos donde se cumple la condición: es igual a 'HOMBRE'
seleccionDeFilas = (Datos['mujer'] == 'HOMBRE')
Datos.loc[seleccionDeFilas, 'mujer'] = 'No'

In [ ]:
Datos['mujer'].value_counts()

In [ ]:
Datos["mujer"] = Datos["mujer"].astype('category')
Datos.dtypes

***
¿Suman las mujeres y hombres iniciales lo mismo que el total después de renombrar?
***

In [ ]:
(8737 + 7981 == 16718) & (8434 + 7864 == 16298)

## 3. Estadísticas descriptivas

### Tablas

In [ ]:
Datos.mean()

In [ ]:
Datos.std()

In [ ]:
Datos.min()

In [ ]:
Datos.max()

In [ ]:
print('Número de observaciones (no missing):')
for col in Datos:
    print(col,'   ', Datos[col].notna().sum() )


### Tablas de frecuencias

In [ ]:
for col in Datos:
    print()
    print('COLUMNA:', col)
    print()
    print(Datos[col].value_counts())


### Gráficos: barras 1

In [ ]:
# Definimos un grupo por los valores de hoy_fuma primero
x1 = Datos.groupby('hoy_fuma').mean().reset_index()
x1

In [ ]:
# Graficamos con variable vertical los números de 0 hasta cuantos grupos haya,
## cruzado con el valor de (el promedio de) edad en cada grupo 
plt.bar(range(len(x1)), x1['edad'])
## Configuraciones gráficas: las etiquetas del eje horizontal y vertical
plt.xticks(range(len(x1)), x1['hoy_fuma'])
plt.ylabel('Promedio de edad')

### Gráficos: barras 2

In [ ]:
# Graficamos con variable vertical los números de 0 hasta cuantos grupos haya,
## cruzado con el valor de (el promedio de) días que no asistió en cada grupo 
plt.bar(range(len(x1)), x1['dias_noasistio'])
## Configuraciones gráficas: las etiquetas del eje horizontal y vertical
plt.xticks(range(len(x1)), x1['hoy_fuma'])
plt.ylabel('Promedio de días')

### Gráficos: boxplot

In [ ]:
plt.boxplot(Datos['t_personas'])

## 4. Regresión

### Correlación

In [ ]:
columnas=['dias_noasistio', 'ha_fumado', 'tercil2016', 'edad', 'mujer', 't_personas']
Datos[columnas].corr()

### Regresión

La columna tercil2016 tiene los siguientes valores:

In [ ]:
# TODO contar cuántos registros tiene cada uno de los valores de tercil2016

**Vamos a crear tres columnas que indiquen si la observación presenta o no cada valor con 1 o 0**

In [ ]:
Datos['tercil20161']=np.NaN
sel = Datos['tercil2016'] == 1
Datos.loc[sel, 'tercil20161'] = 1

sel = Datos['tercil2016'] == 2
Datos.loc[sel, 'tercil20161'] = 0

sel = Datos['tercil2016'] == 3
Datos.loc[sel, 'tercil20161'] = 0

In [ ]:
Datos['tercil20162']=np.NaN
sel = Datos['tercil2016'] == 2
Datos.loc[sel, 'tercil20162'] = 1

sel = Datos['tercil2016'] == 1
Datos.loc[sel, 'tercil20162'] = 0

sel = Datos['tercil2016'] == 3
Datos.loc[sel, 'tercil20162'] = 0

In [ ]:
Datos['tercil20163']=np.NaN
sel = Datos['tercil2016'] == 3
Datos.loc[sel, 'tercil20163'] = 1

sel = Datos['tercil2016'] == 2
Datos.loc[sel, 'tercil20163'] = 0

sel = Datos['tercil2016'] == 1
Datos.loc[sel, 'tercil20163'] = 0

In [ ]:
col=['tercil2016','tercil20161','tercil20162','tercil20163']
Datos[col]

In [ ]:
for var in col:
    Datos[var] = Datos[var].astype('int16')

### El modelo

In [ ]:
mod = sm.OLS.from_formula('dias_noasistio ~ 1 + ha_fumado + tercil20161 + tercil20162 + tercil20163 + edad + mujer + t_personas', data=Datos)
res = mod.fit()
print(res.summary())